# Generate MACD using daily data service - Python

### Overview
*Moving Average Convergence/Divergence Oscillator* was developed by Gerald Appel and represents the difference between two moving averages of different lengths.

Commonly used MACD the 12-day EMA less the 26-day EMA, but the parameters can be adapted when required. As its name implies, MACD is all about convergence and divergence of the two moving averages

MACD oscilaltes around the zero line as follows:
* positive values increase as the shorter moving average diverges further from the longer one
* negative values indicate that the shorter moving average is below the longer one

This sample demonstrates how to request and plot **on-demand** *Moving Average Convergence/Divergence Oscillator - MACD* from a daily data service.

### Inputs/outputs
Moving Average Convergence/Divergence Oscillator's request require instrument's identifier, date time intervals and number of periods of each moving average as per inputs.

*MACD* is computed as follows:

$$ MACD = 12day,EMA - 26day,EMA $$

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve MACD indicator from the hosted service. The queried endpoint in this script are:
* *DailyMacdService*: to directly retrieve daily MACD data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.daily_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package

***

# Run daily MACD sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.daily_analytics.v1.daily_macd_pb2 as daily_macd
import systemathics.apis.services.daily_analytics.v1.daily_macd_pb2_grpc as daily_macd_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *daily macd* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The MACD parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = "AAPL"
exchange = "XNGS"

#### 3.2 MACD parameters

In [ ]:
# set EMAs lengths
long_ema = 26;
short_ema = 12;

#### 3.3 Time period selection

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2019, month = 1, day = 1),
    end_date = date.Date(year = 2020, month = 12, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

#### 3.4 MACD request creation
The following code snippet creates *gRPC client*, process daily MACD requests and returns the reply.

In [ ]:
# generate daily macd request
request = daily_macd.DailyMacdRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    short = short_ema,
    long = long_ema,
    constraints = constraints
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the daily macd service
    service = daily_macd_service.DailyMacdServiceStub(channel)
    
    # Process the daily long macd request
    response = service.DailyMacd(
        request = request, 
        metadata = [('authorization', token)]
    )

### Step 4: Visualize data

#### 4.1 Retrieve MACD and daily prices data
In the following code snippets, the daily MACD and daily prices replies are reprocessed in a dataframe in order to visualize the results:

In [ ]:
# prepare the MACD dataframe
dates=[datetime(m.date.year, m.date.month, m.date.day) for m in response.data]
prices = [m.value for m in response.data]
macd_data = [m.macd for m in response.data]
short_data = [m.short for m in response.data]
long_data = [m.long for m in response.data]

In [ ]:
# create a pandas dataframe with dates, prices, RSIs, short and long EMAs
d = {'Date': dates, 'Macd': macd_data, 'Short': short_data, 'Long': long_data, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot MACD and daily prices
Plot MACD and daily prices request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# add MACD and prices traces
fig.add_trace(go.Scatter(x=df['Date'], y=df['Macd'], name='Macd'), secondary_y=False,)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=True,)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Short'], name='Short EMA'), secondary_y=True,)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Long'], name='Long EMA'), secondary_y=True,)

# add figure title and legend
fig.update_layout(title = 'MACD and daily prices for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Macd', secondary_y=False)
fig.update_yaxes(title_text = 'Price', secondary_y=True)

fig.show()